# ProPyCore SDK Examples: Timecards

This notebook contains snippets from the `timecard` submodule

In [1]:
import os
import dotenv
import json
import ProPyCore

from datetime import datetime

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [2]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [3]:
companies = connection.companies.get()
for company in companies:
    print(company["name"])

company_name = companies[0]["name"]

Rogers-O`Brien Construction


Now get the company details

In [4]:
company = connection.companies.find(identifier=company_name)
projects = connection.projects.get(company_id=company["id"])
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
print(project["name"])

Sandbox Test Project


### `time.timecards.get_time_types()`

```python
"""
Gets the time types at the company level

Parameters
----------
company_id : int
    unique identifier for the company

Returns
-------
<time_types> : list of dict
    list where each value is a dict with a time type's information
"""
```

In [5]:
time_types = connection.time.timecards.get_time_types(
    company_id=company["id"]
)
print(json.dumps(time_types, indent=4))

[
    {
        "id": 980727,
        "name": "Bereavement",
        "abbreviated_time_type": "BER",
        "available": true,
        "default": false,
        "pay_rate": null
    },
    {
        "id": 980729,
        "name": "FMLA",
        "abbreviated_time_type": "FMLA",
        "available": true,
        "default": false,
        "pay_rate": null
    },
    {
        "id": 980894,
        "name": "Hourly",
        "abbreviated_time_type": "REG",
        "available": true,
        "default": false,
        "pay_rate": null
    },
    {
        "id": 89458,
        "name": "Salary",
        "abbreviated_time_type": "SAL",
        "available": true,
        "default": true,
        "pay_rate": null
    },
    {
        "id": 980728,
        "name": "Sick/Personal",
        "abbreviated_time_type": "SICK/PER",
        "available": true,
        "default": false,
        "pay_rate": null
    },
    {
        "id": 44298,
        "name": "Vacation",
        "abbreviated_time_type": "

### `time.timecards.find_time_type()`

In [6]:
salary_time_type = connection.time.timecards.find_time_type(
    company_id=company["id"],
    time_type_identifier="Salary"
)

### `time.timecards.get_for_day()`

```python
"""
Returns a list of all daily timecard data for a given project
https://developers.procore.com/reference/rest/timecard-entries?version=latest

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
entry_date : datetime, default None
    date to pull timecards
    None specifies current day
page : int, default 1
    page number
per_page : int, default 100
    number of timecards to include per page

Returns
-------
timecards : list of dict
    available timecard data
"""
```

In [7]:
timecards_by_day = connection.time.timecards.get_for_day(
    company_id=company["id"],
    project_id=project["id"],
    entry_date=datetime(2025, 2, 5)
)

print(f"Number of Timecards: {len(timecards_by_day)}")
# Uncomment to save timecards
with open("timecards_by_day.json", "w") as f:
    json.dump(timecards_by_day, f, indent=4)

Number of Timecards: 0


### `get_for_specified_period()`
```python
"""
Return a list of all timecard data for the given date range (inclusive on both ends)
https://developers.procore.com/reference/rest/timesheets?version=latest#list-timecard-data

Parameters
----------
company_id : int
    unique identifier for the company
start_date : datetime
    start date of pay period (inclusive)
end_date : datetime
    end date of pay period (inclusive)
page : int, default 1
    page number
per_page : int, default 100
    number of timecards to include per page
party_id : int, default None
    procore People ID to filter by if included

Returns
-------
timecards : list of dict
    available timecard data
"""
```

In [11]:
timecards_by_period = connection.time.timecards.get_for_specified_period(
    company_id=company["id"],
    start_date=datetime(2025, 2, 1),
    end_date=datetime(2025, 2, 5),
    party_id=25138567
)
print(f"Number of Timecards: {len(timecards_by_period)}")
# Uncomment to save timecards
#with open("timecards_by_period.json", "w") as f:
#   json.dump(timecards_by_period, f, indent=4)

Number of Timecards: 4


### `time.timecards.create()`
```python
"""
Create a new timecard
https://developers.procore.com/reference/rest/timecard-entries?version=latest#create-timecard-entry-project

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    Procore's unique identifier for the project
data : dict
    timecard data to create
"""
```

In [9]:
sandbox_project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
data = {
    "hours": 2,                                 # required
    "party_id": "hfritz@r-o.com",               # required: people ID or email (finds the ID for you)
    "date": "2025-02-02",                       # optional: defaults to current day
    "cost_code_id": "Quality Manager",          # required: integer ID or title of cost code (finds ID for you)
    "billable": True,                           # optional: defaults to True
    "description": "Test timecard from API"     # optional
}

entry_data = connection.time.timecards.create(
    company_id=company["id"],
    project_id=sandbox_project["id"],
    data=data
)
print(json.dumps(entry_data,indent=4))

{
    "id": 103771482,
    "approval_status": "pending",
    "approved_by": null,
    "billable": true,
    "clock_in": null,
    "clock_in_time": null,
    "clock_out": null,
    "clock_out_time": null,
    "company_id": 8089,
    "completed_by": null,
    "cost_code": {
        "id": 971232406,
        "biller_id": 2783683,
        "biller_type": "Project",
        "code": ".30.32-",
        "created_at": "2024-07-02T16:08:14Z",
        "deleted_at": null,
        "full_code": "00-.30.32-",
        "long_name": "00-.30.32- - Quality Manager",
        "name": "Quality Manager",
        "parent_id": 971232362,
        "position": null,
        "sortable_code": "00-.30.32-",
        "standard_cost_code_id": 11793993,
        "updated_at": "2024-07-02T16:08:14Z"
    },
    "cost_code_id": 971232406,
    "created_at": "2025-03-17T14:33:17Z",
    "created_by": {
        "id": 9486187,
        "login": "ro-data-connection-0a813c91@procore.com",
        "name": " ro-data-connection-0a813c91"

### `update()`
```python
"""
Updates the given timecard with the provided data
https://developers.procore.com/reference/rest/timecard-entries?version=latest#update-timecard-entry-company

Parameters
----------
company_id : int
    Unique identifier for the company.
project_id : int
    Procore's unique identifier for the project.
timecard_id : int
    Timecard unique identifier.
data : dict
    Timecard data to patch.
"""
```

In [10]:
sandbox_project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
data = {
    "date": "2025-02-02",                       # required
    "hours":9,
    "approval_status": "approved",
    "description": "Updated via API"
}

entry_update = connection.time.timecards.update(
    company_id=company["id"],
    project_id=sandbox_project["id"],
    timecard_id=101882810, # copied from cell above
    data=data
)
print(json.dumps(entry_update,indent=4))

{
    "id": 101882810,
    "approval_status": "approved",
    "approved_by": {
        "id": 9486187,
        "login": "ro-data-connection-0a813c91@procore.com",
        "name": " ro-data-connection-0a813c91"
    },
    "billable": true,
    "clock_in": null,
    "clock_in_time": null,
    "clock_out": null,
    "clock_out_time": null,
    "company_id": 8089,
    "completed_by": null,
    "cost_code": {
        "id": 971232406,
        "biller_id": 2783683,
        "biller_type": "Project",
        "code": ".30.32-",
        "created_at": "2024-07-02T16:08:14Z",
        "deleted_at": null,
        "full_code": "00-.30.32-",
        "long_name": "00-.30.32- - Quality Manager",
        "name": "Quality Manager",
        "parent_id": 971232362,
        "position": null,
        "sortable_code": "00-.30.32-",
        "standard_cost_code_id": 11793993,
        "updated_at": "2024-07-02T16:08:14Z"
    },
    "cost_code_id": 971232406,
    "created_at": "2025-02-06T18:11:28Z",
    "created_by

---